In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"
import tensorflow as tf
import numpy as np
import pandas as pd

In [43]:
# load data
df = pd.read_csv("../csv/numbers.csv")
df.head(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df.shape

(10000, 785)

In [58]:
# Preparar datos
def one_hot(n_value, num_classes=10):
    l_arr = np.zeros(num_classes)
    if 0 <= int(n_value) < num_classes:
        l_arr[int(n_value)] = 1
    else:
        print(f"Advertencia: Valor de etiqueta {n_value} fuera de rango para {num_classes} clases.")
    return l_arr

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Y_raw = df.iloc[:, 0].values # todas las filas [:] columna 0 (labels) [:, 0]
X_raw = df.iloc[:, 1:].values # todas las filas [:] colunas de la 1 en adelante (datos) [:, 1:]
Y_processed = np.array([one_hot(n) for n in Y_raw]).astype('float32')
x_train = x_train.astype('float32') / 255.0 # para normalizar los datos 0-1
y_train = np.array([one_hot(n) for n in y_train]).astype('float32')
X_normalized = X_raw.astype('float32') / 255.0 # para normalizar los datos 0-1

x_train = x_train.reshape(60000, 784)
X_normalized.shape, Y_processed.shape, x_train.shape, y_train.shape



((10000, 784), (10000, 10), (60000, 784), (60000, 10))

In [ ]:
# Crear Modelo
model = tf.keras.Sequential(name="myMnist")
model.add(tf.keras.layers.InputLayer(shape=(784,)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Resumen del modelo
model.summary()


Model: "myMnist"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_33 (Dense)                │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,210 (231.29 KB)

 Trainable params: 59,210 (231.29 KB)

 Non-trainable params: 0 (0.00 B)

In [62]:
# Entrenar modelo
print("\nEntrenando con Keras...")
pasos = 200
batch = 32
# history = model.fit(X_normalized, Y_processed, epochs=pasos, batch_size=batch, verbose=1)
history = model.fit(x_train, y_train, epochs=pasos, batch_size=batch, verbose=1)

print(f"Pérdida final: {history.history['loss'][-1]:.4f}")
print(f"Accuracy final: {history.history['accuracy'][-1]:.4f}")

# Puedes guardar y cargar el modelo fácilmente
model.save("mnist.h5")
# loaded_model = tf.keras.models.load_model("mi_modelo_keras.h5")


Entrenando con Keras...
Epoch 1/200


2025-05-28 16:12:55.146255: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8733 - loss: 0.4154
Epoch 2/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9476 - loss: 0.1985
Epoch 3/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9544 - loss: 0.1756
Epoch 4/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9603 - loss: 0.1576
Epoch 5/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9636 - loss: 0.1463
Epoch 6/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9688 - loss: 0.1324
Epoch 7/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9693 - loss: 0.1287
Epoch 8/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9712 - loss: 0.1254
Epoch 9/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9709 - loss: 0.1219
Epoch 10/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9747 - loss: 0.1089
Epoch 11/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9731 - loss: 0.1181
Epoch 12/200
1875/1875 ━━━━━━━━

Pérdida final: 0.2669
Accuracy final: 0.8789
